In [1]:
import json

with open("train_item.json") as training:
    training = json.loads(training.read())
    
with open("test_item.json") as testing:
    testing = json.loads(testing.read())

In [2]:
training_dict = {}
testing_dict = {}

for user in training.keys():
    for [item, rating] in training[user]:
        if user in training_dict:
            training_dict[user][item] = rating
        else:
            training_dict[user] = {item: rating}
            
for user in testing.keys():
    for [item, rating] in testing[user]:
        if user in testing_dict:
            testing_dict[user][item] = rating
        else:
            testing_dict[user] = {item: rating}

# print(training_dict['A12R54MKO17TW0'])

In [3]:
def sub_row_mean(ratings_dict):
    sub_row_mean_ratings_dict = {}
    for user in ratings_dict:
        list_item = list(ratings_dict[user].values())
        avg = sum(list_item) / len(list_item)
        for item in ratings_dict[user].keys():
            if user in sub_row_mean_ratings_dict:
                sub_row_mean_ratings_dict[user][item] = ratings_dict[user][item] - avg
            else:
                sub_row_mean_ratings_dict[user] = {item: ratings_dict[user][item] - avg}
    
    return sub_row_mean_ratings_dict
        
# training_dict2 = sub_row_mean(training_dict)
# print(training_dict2['A12R54MKO17TW0'])
# print(training_dict['A12R54MKO17TW0'])

In [4]:
import numpy as np
from numpy.linalg import norm
import math

def cosine_sim(ratings_dict, user):
    cosine_sim_val = {}

    for other_user in ratings_dict.keys():
        dot_prod = sum(ratings_dict[user][item]*ratings_dict[other_user].get(item, 0) for item in ratings_dict[user])
        user_norm = np.linalg.norm(np.array([rating for rating in ratings_dict[user].values()]))
        other_user_norm = np.linalg.norm(np.array([rating for rating in ratings_dict[other_user].values()]))
        norm_mul = user_norm * other_user_norm
        if norm_mul == 0:
            cos_sim = 0
        else:
            cos_sim = dot_prod / norm_mul
        if math.isnan(cos_sim):
            cos_sim = 0
        cosine_sim_val[other_user] = cos_sim
    
        if cosine_sim_val[other_user] > 1:
            cosine_sim_val[other_user] = 1
        if cosine_sim_val[other_user] < 0:

            cosine_sim_val[other_user] = 0        
    
    return cosine_sim_val

In [5]:

def predict_item_rating_avg(ratings_dict, cos_similarities_dict, k, item, user):

    cos_similarities_dict.pop(user)
    other_users = list(cos_similarities_dict.keys())
    for other_user in other_users:
        if item not in ratings_dict[other_user]:
            cos_similarities_dict.pop(other_user)
    
    list_cos_sim = sorted(cos_similarities_dict.items(), key=lambda x: x[1], reverse=True)[:k]

    avg_rating = 0
    
    if len(list_cos_sim) != 0 and list_cos_sim[0][1] != 0:
        for pair in list_cos_sim:
            avg_rating += ratings_dict[pair[0]][item]
        
        avg_rating = avg_rating / len(list_cos_sim)
    else:
        total = 0
        rating = 0
        for rat in training_dict[user]:
            rating += training_dict[user][rat]
            total += 1
        rating = rating / total
        return rating
    
    return avg_rating


    
    

In [6]:


def predict_item_rating_sim(ratings_dict, cos_similarities_dict, k, item, user):

    if user in cos_similarities_dict:
        cos_similarities_dict.pop(user)
    other_users = list(cos_similarities_dict.keys())
    for other_user in other_users:
        if item not in ratings_dict[other_user]:
            cos_similarities_dict.pop(other_user)
    
    list_cos_sim = sorted(cos_similarities_dict.items(), key=lambda x: x[1], reverse=True)[:k]
    rating = 0
    similarities = 0
    
    if len(list_cos_sim) != 0 and list_cos_sim[0][1] != 0:
        for pair in list_cos_sim:
            rating += ratings_dict[pair[0]][item] * cos_similarities_dict[pair[0]]
            similarities += cos_similarities_dict[pair[0]]
        rating = rating / similarities
    else:
        total = 0
        rating = 0
        for rat in training_dict[user]:
            rating += training_dict[user][rat]
            total += 1
        rating = rating / total
        
    if rating > 5.1:
        print("Flag")
        print("")
        rating = 4.99
    return rating

  
    

In [7]:
def predict(ratings_dict, user, items):
    predicted_ratings_dict = {}
    sub_mean_dict = sub_row_mean(ratings_dict)
    cos_sim_dict = cosine_sim(sub_mean_dict, user)
    for item in items:
        avg_rating_prediction = predict_item_rating_avg(training_dict, cos_sim_dict.copy(), k, item, user)
        sim_rating_prediction = predict_item_rating_sim(training_dict, cos_sim_dict.copy(), k, item, user)
        
        predicted_ratings_dict[item] = (avg_rating_prediction, sim_rating_prediction)
        
    return predicted_ratings_dict

In [8]:
def first_one_in_dict(dict_first_one):
    count = 0
    for item,rating in dict_first_one.items():
        if rating == 1:
            return count
        else:
            count+=1
    return -1


In [9]:
def ndcg(recommended_items_ratings_avg, recommended_items_ratings_sim, user):
    orig_reviewed_items = (sorted(testing_dict[user].items(), key=lambda x: x[1], reverse=True)).keys()
    orig_reviewed_items_hit_avg = {}
    orig_reviewed_items_hit_sim = {}
    
    for item in orig_reviewed_items:
        orig_reviewed_items_hit_avg[item] = 0
        orig_reviewed_items_hit_sim[item] = 0
    
    recommended_items_avg = []#recommended_items_ratings_avg.keys()
    recommended_items_sim = []#recommended_items_ratings_sim.keys()

    for item in recommended_items_ratings_avg:
        recommended_items_avg.append(item[0])

    for item in recommended_items_ratings_sim:
        recommended_items_sim.append(item[0])

    same_avg_items = 0
    same_sim_items = 0

    for item in recommended_items_avg:
        if item in orig_reviewed_items:
            same_avg_items += 1
            orig_reviewed_items_hit_avg[item] = 1
            
            

    for item in recommended_items_sim:
        if item in orig_reviewed_items:
            same_sim_items += 1
            orig_reviewed_items_hit_sim[item] = 1
            
    dcg_avg = 0
    idcg_avg = 0
    ndcg_avg = 0
    if same_avg_items == 0:
        ndcg_avg = 0
    else:
        first_one = first_one_in_dict(orig_reviewed_items_hit_avg)
        for i in range(same_avg_items):
            dcg_avg += (1/math.log2(1 + first_one + i + 1))
            idcg_avg += (1/math.log2(1 + i + 1))
            
        ndcg_avg = dcg_avg/idcg_avg
        
        
    dcg_sim = 0
    idcg_sim = 0
    ndcg_sim = 0
    if same_sim_items == 0:
        ndcg_sim = 0
    else:
        first_one = first_one_in_dict(orig_reviewed_items_hit_sim)
        for i in range(same_sim_items):
            dcg_sim += (1/math.log2(1 + first_one + i + 1))
            idcg_sim += (1/math.log2(1 + i + 1))
            
        ndcg_sim = dcg_sim/idcg_sim
    print('NDCG Avg: ')
    print(ndcg_avg)
    print('NDCG Sim: ')
    print(ndcg_sim)
    #return (ndcg_avg, ndcg_sim)     
    


In [10]:
def scores(recommended_items_ratings_avg, recommended_items_ratings_sim, user):

#     recommended_items_ratings_avg = recommended_items_avg
#     recommended_items_ratings_sim = recommended_items_avg

    print((user, recommended_items_ratings_avg))
    print((user, recommended_items_ratings_sim))

    orig_reviewed_items = testing_dict[user].keys()
    recommended_items_avg = []#recommended_items_ratings_avg.keys()
    recommended_items_sim = []#recommended_items_ratings_sim.keys()
    recommended_items_pair_avg = {}#recommended_items_ratings_avg.keys()
    recommended_items_pair_sim = {}#recommended_items_ratings_sim.keys()
    

    for item in recommended_items_ratings_avg:
        recommended_items_avg.append(item[0])
        recommended_items_pair_avg[item[0]] = 0 
        

    for item in recommended_items_ratings_sim:
        recommended_items_sim.append(item[0])
        recommended_items_pair_sim[item[0]] = 0 

    same_avg_items = 0
    same_sim_items = 0

    for item in recommended_items_avg:
        if item in orig_reviewed_items:
            same_avg_items += 1
            recommended_items_pair_avg[item] = 1

    for item in recommended_items_sim:
        if item in orig_reviewed_items:
            same_sim_items += 1
            recommended_items_pair_sim[item] = 1

    precision_avg = (same_avg_items/k)
    precision_sim = (same_sim_items/k)

    print("Precision Avg: ")
    print(precision_avg)
    print('Precision Sim: ')
    print(precision_sim)


    recall_avg = (same_avg_items/len(orig_reviewed_items))
    recall_sim = (same_sim_items/len(orig_reviewed_items))

    print('Recall Avg: ')
    print(recall_avg)
    print('Recall Sim: ')
    print(recall_sim)
    if precision_avg+recall_avg == 0:
        f_score_avg = 0
    else:
        f_score_avg = (2*precision_avg*recall_avg)/(precision_avg+recall_avg)

    if precision_sim+recall_sim == 0:
        f_score_sim = 0
    else:
        f_score_sim = (2*precision_sim*recall_sim)/(precision_sim+recall_sim)

    print('F_Score Avg: ')
    print(f_score_avg)
    print('F_Score Sim: ')
    print(f_score_sim)
    print(testing_dict[user])
    
    
    #ndcg(recommended_items_ratings_avg, recommended_items_ratings_sim, user)
    
    dcg_avg = 0
    idcg_avg = 0
    ndcg_avg = 0
    if same_avg_items != 0:
        first_one = first_one_in_dict(recommended_items_pair_avg)
        for i in range(same_avg_items):
            dcg_avg += (1/math.log2(1 + first_one + i + 1))
            idcg_avg += (1/math.log2(1 + i + 1))
            
        ndcg_avg = dcg_avg/idcg_avg
        
        
    dcg_sim = 0
    idcg_sim = 0
    ndcg_sim = 0
    if same_sim_items != 0:
        first_one = first_one_in_dict(recommended_items_pair_sim)
        for i in range(same_sim_items):
            dcg_sim += (1/math.log2(1 + first_one + i + 1))
            idcg_sim += (1/math.log2(1 + i + 1))
            
        ndcg_sim = dcg_sim/idcg_sim
    print('NDCG Avg: ')
    print(ndcg_avg)
    print('NDCG Sim: ')
    print(ndcg_sim)
    
    return(precision_avg, precision_sim, recall_avg, recall_sim, f_score_avg, f_score_sim, ndcg_avg, ndcg_sim)
    #return (ndcg_avg, ndcg_sim)     
    

    
    
    
    
    
    
    
    

In [11]:
all_items = set()
for user,list_ratings in training_dict.items():
    all_items.update(list(training_dict[user].keys()))

for user,list_ratings in testing_dict.items():
    all_items.update(list(testing_dict[user].keys()))
#print(len(all_items))

In [12]:
top_n_testing_dict = dict(sorted(testing_dict.items(), key=lambda x: len(x[1]), reverse=True)[:100])

In [13]:
#top_n_testing_dict

In [14]:
import time

k = 10
# item = ["B0048P1XXQ"]
#user = "A12R54MKO17TW0"

prec_avg, prec_sim, rec_avg, rec_sim, f_avg, f_sim, ndcgav, ndcgsim = 0, 0, 0, 0, 0, 0, 0, 0

start = time.time()
i = 0
j = 0

for user, list_ratings in top_n_testing_dict.items():
    i += 1
    predicted_vals = predict(training_dict, user, list(all_items))

    orig_vals = training_dict[user]

    for item,rating in orig_vals.items():

        predicted_vals.pop(item)
        
    print('User Count: ' + str(i))
    print(time.time() - start)


    recommended_items_ratings_avg = sorted(predicted_vals.items(), key=lambda x: x[1][0], reverse=True)[:k]
    recommended_items_ratings_sim = sorted(predicted_vals.items(), key=lambda x: x[1][1], reverse=True)[:k]

    temp_prec_avg, temp_prec_sim, temp_rec_avg, temp_rec_sim, temp_f_avg, temp_f_sim, temp_ndcgav, temp_ndcgsim = scores(recommended_items_ratings_avg, recommended_items_ratings_sim, user)
    
    prec_avg += temp_prec_avg
    prec_sim += temp_prec_sim
    rec_avg += temp_rec_avg
    rec_sim += temp_rec_sim
    f_avg += temp_f_avg
    f_sim += temp_f_sim
    ndcgav += temp_ndcgav
    ndcgsim += temp_ndcgsim




end = time.time()

print(end - start) #=> 0.0011411383748054504

print('Total Precision Average: ')
print(prec_avg/100)
print('Total Precision Similar: ')
print(prec_sim/100)
print('Total Recall Average: ')
print(rec_avg/100)
print('Total Recall Similar: ')
print(rec_sim/100)
print('Total F Score Average: ')
print(f_avg/100)
print('Total F Score Similar: ')
print(f_sim/100)
print('NDCG Average: ')
print(ndcgav/100)
print('NDCG Similar: ')
print(ndcgsim/100)

User Count: 1
890.6262261867523
('B00CZF8B68', [('A11KOVAKQWQ3RF', (5.0, 5.0)), ('AFUIT5NVTFSG7', (5.0, 5.0)), ('ASM3K6OK2QGIO', (5.0, 5.0)), ('AEQYI98EIDRV2', (5.0, 5.0)), ('AZ9NC91NEBZPO', (5.0, 5.0)), ('A2PXHB1X94548M', (5.0, 5.0)), ('A2TXOIMFCMYEY1', (5.0, 5.000000000000001)), ('A2IU16H9LFL593', (5.0, 5.0)), ('A2QTN0MB63MF6T', (5.0, 5.0)), ('ADQ0H3OSW3H49', (5.0, 5.0))])
('B00CZF8B68', [('A2U9TMU40ZJWBL', (5.0, 5.000000000000003)), ('A2NRH0W1INVZV4', (5.0, 5.000000000000003)), ('A3AJIQEVA6T0GQ', (5.0, 5.000000000000003)), ('A2R7E8HSE7NQEO', (5.0, 5.000000000000002)), ('A1FPFWRPLT3DAG', (5.0, 5.000000000000002)), ('A2ZNNL33ULQ7VV', (4.857142857142857, 5.000000000000002)), ('A2YXQJ9VSOKWBZ', (5.0, 5.000000000000002)), ('A2M4FLWO2IG3AY', (5.0, 5.000000000000002)), ('AJI7Q53V06VPR', (5.0, 5.000000000000002)), ('A3FNKUFTIN71WI', (5.0, 5.000000000000002))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A1US7BFCSQKQ70': 5.0, 'A

User Count: 3
2326.4397435188293
('B00BWGHIHY', [('AFUIT5NVTFSG7', (5.0, 5.000000000000001)), ('A35RUNJMLQJTUG', (5.0, 5.0)), ('A2PXHB1X94548M', (5.0, 5.0)), ('A1UDC1O6SS2RF', (5.0, 5.0)), ('A2TXOIMFCMYEY1', (5.0, 5.0)), ('A3AMN8YYXCUA7W', (5.0, 5.0)), ('AUJHH0GA8BI5R', (5.0, 5.0)), ('A1TATEICW2UWK9', (5.0, 5.0)), ('A2IA0VM159I4GZ', (5.0, 5.0)), ('AP87DTP6TO6XN', (5.0, 5.0))])
('B00BWGHIHY', [('A2F4X5H87BGPP3', (5.0, 5.000000000000002)), ('A5TLQ5EQIJQYT', (5.0, 5.000000000000002)), ('A2AVENA0EZ2G8K', (5.0, 5.000000000000002)), ('A225KOTGHS3BRA', (5.0, 5.000000000000002)), ('A3DYLNS3B1ZUUO', (5.0, 5.000000000000002)), ('A1HVVENWQI9VQB', (5.0, 5.000000000000002)), ('A1LVZI3QBCW9A0', (5.0, 5.000000000000002)), ('A161A2BZ58MUNJ', (5.0, 5.000000000000002)), ('A281DXZBJSUHPQ', (5.0, 5.000000000000002)), ('AN9PM7Q7ZMZWH', (5.0, 5.000000000000002))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A23XDLRMGN5BCS': 5.0, 'A1YL3G7W55HBXG

User Count: 6
4165.143839836121
('B00EH49FRE', [('AFUIT5NVTFSG7', (5.0, 4.999999999999999)), ('A1UDC1O6SS2RF', (5.0, 5.0)), ('A2IU16H9LFL593', (5.0, 5.0)), ('A3PFYHPEL3KIJR', (5.0, 5.0)), ('ADQ0H3OSW3H49', (5.0, 5.0)), ('A12D0AWWR30CGF', (5.0, 5.0)), ('A1ZNX5OOYR356Z', (5.0, 5.0)), ('A3AMN8YYXCUA7W', (5.0, 5.000000000000001)), ('A1TATEICW2UWK9', (5.0, 4.999999999999999)), ('A1DFZM3OOO5XGZ', (5.0, 5.0))])
('B00EH49FRE', [('A35P1COO95IH1R', (5.0, 5.000000000000003)), ('A2R7PJGG0BWY0J', (5.0, 5.000000000000003)), ('AGTM2VNAL7VS3', (5.0, 5.000000000000003)), ('A3G6G5BJ67W1KP', (5.0, 5.000000000000002)), ('AVH5I2EQV6TH2', (5.0, 5.000000000000002)), ('A1NDKH5MGL8MR1', (5.0, 5.000000000000002)), ('A3DBDULLZISWMC', (4.8, 5.000000000000002)), ('A1YVCC7QNAGU01', (4.333333333333333, 5.000000000000002)), ('A260VHH71O75UL', (5.0, 5.000000000000002)), ('A32T2H8150OJLU', (5.0, 5.000000000000002))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 

User Count: 9
5991.389736890793
('B0092MKTL2', [('A11KOVAKQWQ3RF', (5.0, 5.0)), ('AEQYI98EIDRV2', (5.0, 5.0)), ('A3PFYHPEL3KIJR', (5.0, 5.0)), ('A1ZNX5OOYR356Z', (5.0, 5.0)), ('AUJHH0GA8BI5R', (5.0, 5.0)), ('AOE1NDE8169WG', (5.0, 5.0)), ('A1QENYMHRFWY0T', (5.0, 4.999999999999999)), ('AKMWILI3V2XCY', (5.0, 5.0)), ('A3NHU3WTO3RXIR', (5.0, 5.0)), ('A1KNCVDPEF6TJ8', (5.0, 5.0))])
('B0092MKTL2', [('ABCXJIK7QMLA7', (5.0, 5.000000000000003)), ('A35GU60F0WQJ7B', (4.833333333333333, 5.000000000000002)), ('A1GR5SRP9SLDPA', (5.0, 5.000000000000002)), ('A3T13H29NQEJOI', (5.0, 5.000000000000002)), ('AS0CCKP5ZB0M6', (5.0, 5.000000000000002)), ('A1F37SJMP29EMF', (4.888888888888889, 5.000000000000002)), ('A3GYWZE27QBMY7', (5.0, 5.000000000000002)), ('A6XMXS05BQ6MK', (5.0, 5.000000000000002)), ('AZG22STYRLSYZ', (5.0, 5.000000000000001)), ('A153OTHBFLY65L', (5.0, 5.000000000000001))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A1X2S2BF67JJ

User Count: 13
8436.991765260696
('B0092MKTWQ', [('AFUIT5NVTFSG7', (5.0, 4.999999999999999)), ('ASM3K6OK2QGIO', (5.0, 5.0)), ('AEQYI98EIDRV2', (5.0, 5.0)), ('A2PXHB1X94548M', (5.0, 4.999999999999999)), ('A3PFYHPEL3KIJR', (5.0, 5.000000000000001)), ('A2QTN0MB63MF6T', (5.0, 5.0)), ('A3AMN8YYXCUA7W', (5.0, 5.0)), ('AUJHH0GA8BI5R', (5.0, 5.0)), ('A391EYZCKZ1TCC', (5.0, 5.0)), ('A2IA0VM159I4GZ', (5.0, 5.0))])
('B0092MKTWQ', [('A6XHF110BC856', (5.0, 5.000000000000003)), ('A20173T9HDB759', (5.0, 5.000000000000003)), ('A39VP8I0ATNCB1', (5.0, 5.000000000000002)), ('A5PZNLSCMM4KQ', (5.0, 5.000000000000002)), ('A2FNSPM7N4V401', (5.0, 5.000000000000002)), ('A1BAZSHD67PTWO', (5.0, 5.000000000000002)), ('A1G5XXV19J68WG', (5.0, 5.000000000000002)), ('ASABX35QYUEZL', (5.0, 5.000000000000002)), ('A1LUGVFDKPE7N7', (5.0, 5.000000000000002)), ('A26YREV8F03L2Q', (5.0, 5.000000000000002))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'AAFGG6R921

...

User Count: 33
21325.153651237488
('B00330WKMA', [('AFUIT5NVTFSG7', (5.0, 5.0)), ('ASM3K6OK2QGIO', (5.0, 5.0)), ('AZ9NC91NEBZPO', (5.0, 5.0)), ('A35RUNJMLQJTUG', (5.0, 5.0)), ('A2PXHB1X94548M', (5.0, 5.0)), ('A2QTN0MB63MF6T', (5.0, 5.0)), ('A1ZNX5OOYR356Z', (5.0, 5.0)), ('A3AMN8YYXCUA7W', (5.0, 5.0)), ('A2QLXV5IRNDTA6', (5.0, 5.0)), ('A11TSWD8UXE62A', (5.0, 5.0))])
('B00330WKMA', [('A2YXQJ9VSOKWBZ', (5.0, 5.000000000000002)), ('A1AMAN52V34BR7', (5.0, 5.000000000000002)), ('AYGJ69VWZWS1W', (5.0, 5.000000000000002)), ('A11JTCOUI4KA00', (4.6, 5.000000000000002)), ('A1NQOJEXEAX0PS', (5.0, 5.000000000000002)), ('A320RC9IY52X8T', (5.0, 5.000000000000002)), ('A3UL0IZE11VU8A', (5.0, 5.000000000000002)), ('A3BZIXUPWNY4HH', (5.0, 5.000000000000002)), ('A3QE3KFPKO0VCR', (5.0, 5.000000000000002)), ('AZ3BUR9RQBZ0O', (5.0, 5.000000000000002))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A2X8HNENH4ZMUI': 5.0, 'A6DSDTFL9O4HB': 5.0, 'A1LM

User Count: 37
23761.638290166855
('B008XCJ0KI', [('A11KOVAKQWQ3RF', (5.0, 5.0)), ('AFUIT5NVTFSG7', (5.0, 5.0)), ('ASM3K6OK2QGIO', (5.0, 5.000000000000001)), ('AEQYI98EIDRV2', (5.0, 5.0)), ('A2PXHB1X94548M', (5.0, 5.0)), ('A2TXOIMFCMYEY1', (5.0, 5.0)), ('A2IU16H9LFL593', (5.0, 5.0)), ('A3PFYHPEL3KIJR', (5.0, 5.0)), ('A1ZNX5OOYR356Z', (5.0, 5.0)), ('AUJHH0GA8BI5R', (5.0, 5.0))])
('B008XCJ0KI', [('A13G2LMKA9GVMM', (5.0, 5.000000000000003)), ('A3TQTXHGL2WGR5', (5.0, 5.000000000000003)), ('AQQI92H5SN380', (5.0, 5.000000000000002)), ('A12XZ6VT9XMFNE', (5.0, 5.000000000000002)), ('A3GRNRNGUSFNNN', (5.0, 5.000000000000002)), ('A324G7JXU9Y57G', (5.0, 5.000000000000002)), ('ABRHNTJ0IJH7L', (5.0, 5.000000000000002)), ('A2U9TMU40ZJWBL', (5.0, 5.000000000000002)), ('A19GQNQLVEAGH0', (5.0, 5.000000000000002)), ('A1XV95ID4PBQSH', (5.0, 5.000000000000002))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A1LVEPRENTHRSH': 5.0, 'A2TVPIDNOLDYX

User Count: 41
26196.742247104645
('B009KUAOH0', [('AFUIT5NVTFSG7', (5.0, 5.0)), ('ASM3K6OK2QGIO', (5.0, 5.0)), ('AEQYI98EIDRV2', (5.0, 5.000000000000001)), ('AZ9NC91NEBZPO', (5.0, 5.0)), ('A2PXHB1X94548M', (5.0, 5.0)), ('A2VEDIT2AJCQSX', (5.0, 5.0)), ('A3PFYHPEL3KIJR', (5.0, 5.0)), ('ADQ0H3OSW3H49', (5.0, 5.0)), ('AUJHH0GA8BI5R', (5.0, 5.0)), ('A1TATEICW2UWK9', (5.0, 5.0))])
('B009KUAOH0', [('A3AZ4L6OS6KFZU', (5.0, 5.000000000000002)), ('A50N0VSCHXJQD', (5.0, 5.000000000000002)), ('ADHZ0ABZBBDFJ', (5.0, 5.000000000000002)), ('A53RCDJ4OJW5L', (5.0, 5.000000000000002)), ('A32CCSLZMFN020', (5.0, 5.000000000000002)), ('AO7IDK5D75WO6', (5.0, 5.000000000000002)), ('A1QIZFFDOSBDTA', (5.0, 5.000000000000002)), ('A73LKV5UNIX73', (5.0, 5.000000000000002)), ('A33H2FCAJE6W8K', (5.0, 5.000000000000002)), ('A3L8YUEWQM3K5K', (5.0, 5.000000000000002))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A2OS4FCI886UXB': 5.0, 'A271O9APH0W3JP': 5

User Count: 46
29231.04972243309
('B00B113E0U', [('A11KOVAKQWQ3RF', (5.0, 5.000000000000001)), ('AFUIT5NVTFSG7', (5.0, 5.000000000000001)), ('ASM3K6OK2QGIO', (5.0, 5.0)), ('A1DFQC66IOU5UX', (5.0, 5.0)), ('A1ZNX5OOYR356Z', (5.0, 5.000000000000001)), ('A373ETRP1DHG9M', (5.0, 5.0)), ('A1JTUF889JJSYF', (5.0, 5.0)), ('A3034VFEK52CU4', (5.0, 5.0)), ('A19YSJHAUCRV6N', (5.0, 5.0)), ('A26UU6E4MRI4PN', (5.0, 5.0))])
('B00B113E0U', [('AXLWYRXQE0FJU', (5.0, 5.000000000000002)), ('A11KOVAKQWQ3RF', (5.0, 5.000000000000001)), ('AFUIT5NVTFSG7', (5.0, 5.000000000000001)), ('A1ZNX5OOYR356Z', (5.0, 5.000000000000001)), ('AQKVDFXFDX47B', (5.0, 5.000000000000001)), ('A32447BB4SG8XZ', (5.0, 5.000000000000001)), ('A1GYY9CWGSM2F1', (5.0, 5.000000000000001)), ('A11QTEA0XNK0WW', (5.0, 5.000000000000001)), ('AA5HPTKCEJWZ', (5.0, 5.000000000000001)), ('A14EHD9457KKXY', (5.0, 5.000000000000001))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A2GRTK0AQB

KeyboardInterrupt: 